# Module 3: Data Retrieval with SQL (Farmer's Market Database)

This notebook demonstrates key SQL data retrieval operations using Python and MySQL.
We will use the Farmer’s Market database hosted locally on a MySQL server.
Each section includes explanations and annotated code examples to reinforce learning.

Before running this notebook, ensure:
- MySQL Server is running on `localhost`
- The `farmers_market` database is already imported
- Python packages `mysql-connector-python` and `pandas` are installed


In [ ]:
# Install dependencies if needed (uncomment and run once)
# !pip install mysql-connector-python pandas


## Step 1: Connect to MySQL Server

We use `mysql.connector` to connect to the local MySQL instance.
Please replace the placeholders for `user` and `password` with your actual credentials.


In [216]:

import mysql.connector
import pandas as pd

# Establish connection to local MySQL server
conn = mysql.connector.connect(
    host="localhost",
    user="root",       # very bad practice, never use root in production, never hardcode credentials!
    password="William2025!!",   # Replace with your MySQL password
    database="farmers_market"
)

cursor = conn.cursor()
print("Succesfully Connected to MySQL database!")



Succesfully Connected to MySQL database!


In [217]:
# Example: Select all columns from the customer table. Not wise for production use, but good for learning.
query = '''
SELECT * 
FROM farmers_market.customer;
'''

cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
df.head()

,customer_id,customer_first_name,customer_last_name,customer_zip
0,1,Jane,Connor,22801
1,2,Manuel,Diaz,22821
2,3,Bob,Wilson,22821
3,4,Deanna,Washington,22801
4,5,Abigail,Harris,22801


In [218]:
# Example: Select a few specific columns from the vendor table
query = '''
SELECT 
    vendor_id,
    vendor_name,
    vendor_type
FROM farmers_market.vendor;
'''
 

cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
df.head()

,vendor_id,vendor_name,vendor_type
0,1,Chris's Sustainable Eggs & Meats,Eggs & Meats
1,2,Hernández Salsa & Veggies,Fresh Variety: Veggies & More
2,3,Mountain View Vegetables,Fresh Variety: Veggies & More
3,4,Fields of Corn,Fresh Focused
4,5,Seashell Clay Shop,Arts & Jewelry


## Filtering with WHERE Clause

The `WHERE` clause filters rows based on specific conditions.



In [219]:
query = '''
SELECT 
    product_id,
    product_name,
    product_qty_type,
    product_category_id
FROM farmers_market.product
WHERE product_category_id > 3;
'''

cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
df

,product_id,product_name,product_qty_type,product_category_id
0,6,Cut Zinnias Bouquet,unit,5
1,10,Eggs,unit,6
2,11,Pork Chops,lbs,6
3,19,Farmer's Market Resuable Shopping Bag,unit,7
4,20,Homemade Beeswax Candles,unit,7


## ORDER BY and LIMIT Clauses

- `ORDER BY` is used to sort the result set by one or more columns.
- `LIMIT` restricts the number of returned rows.

In [220]:
query = '''
SELECT
    vendor_name,
    vendor_type
FROM farmers_market.vendor
ORDER BY vendor_name 
LIMIT 5;
'''

cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
df


,vendor_name,vendor_type
0,Annie's Pies,Prepared Foods
1,Chris's Sustainable Eggs & Meats,Eggs & Meats
2,Fields of Corn,Fresh Focused
3,Hernández Salsa & Veggies,Fresh Variety: Veggies & More
4,Marco's Peppers,Fresh Focused


##  Inline Functions

Use inline functions to perform calculations directly in SQL.

```

In [221]:
query = '''
SELECT
    product_id,
    customer_id,
    cost_to_customer_per_qty,
    ROUND(cost_to_customer_per_qty, 1) AS rounded_cost_to_customer_per_qty
FROM farmers_market.customer_purchases
LIMIT 6;
'''

# Use a fresh cursor
cursor = conn.cursor()
cursor.execute(query)

# Fetch and close safely
results = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
cursor.close()

# Load to DataFrame

df = pd.DataFrame(results, columns=columns)
df


,product_id,customer_id,cost_to_customer_per_qty,rounded_cost_to_customer_per_qty
0,1,14,6.99,7.0
1,1,14,6.99,7.0
2,1,15,6.99,7.0
3,1,16,6.99,7.0
4,1,22,6.99,7.0
5,1,4,6.99,7.0


In [228]:
query = '''
SELECT 
    customer_id,
    product_id,
    vendor_id,
    quantity,
    cost_to_customer_per_qty,
    quantity * cost_to_customer_per_qty AS total_cost
FROM farmers_market.customer_purchases
WHERE quantity * cost_to_customer_per_qty > 100;
'''

cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()
cursor.close()

# Optionally load into pandas

df = pd.DataFrame(results, columns=['customer_id', 'product_id', 'vendor_id', 'quantity', 'cost', 'total_cost'])
df


,customer_id,product_id,vendor_id,quantity,cost,total_cost
0,21,7,8,6.00,18.00,108.0000
1,22,8,8,8.00,18.00,144.0000
2,24,8,8,6.00,18.00,108.0000


## String Functions: CONCAT

You can merge string columns using `CONCAT()`.

**Example:**
```sql
SELECT CONCAT(first_name, ' ', last_name) AS full_name FROM ...
```

In [223]:
query = '''
SELECT
    CONCAT(vendor_owner_first_name," " ,vendor_owner_last_name) AS vendor_name,
    vendor_type
FROM farmers_market.vendor
LIMIT 5;
'''

# Create a fresh cursor
cursor = conn.cursor()
cursor.execute(query)

# Fetch all results and get column headers
results = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

# Always close the cursor after use
cursor.close()

# Load into DataFrame
import pandas as pd
df = pd.DataFrame(results, columns=columns)
df


,vendor_name,vendor_type
0,Chris Sylvan,Eggs & Meats
1,Maria Hernández,Fresh Variety: Veggies & More
2,Joseph Yoder,Fresh Variety: Veggies & More
3,Samuel Smith,Fresh Focused
4,Karen Soula,Arts & Jewelry


## Wrap Up

We’ve covered:

- SELECT statements
- Filtering with WHERE
- Sorting and limiting results
- Inline arithmetic
- String concatenation

In [ ]:
# Don’t forget to close your DB connection after use
cursor.close()
conn.close()
print(" MySQL connection closed.")